In [74]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec


In [75]:
data = pd.read_csv('../Data/assignment2.csv')

In [76]:
data.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [77]:
data.describe(include='all')

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
count,11914,11914,11914.000000,11911,11845.00000,11884.000000,11914,11914,11908.000000,8172,11914,11914,11914.000000,11914.000000,11914.000000,1.191400e+04
unique,48,915,NaN,10,NaN,NaN,5,4,NaN,71,3,16,NaN,NaN,NaN,NaN
top,Chevrolet,Silverado 1500,NaN,regular unleaded,NaN,NaN,AUTOMATIC,front wheel drive,NaN,Crossover,Compact,Sedan,NaN,NaN,NaN,NaN
freq,1123,156,NaN,7172,NaN,NaN,8266,4787,NaN,1110,4764,3048,NaN,NaN,NaN,NaN
mean,NaN,NaN,2010.384338,NaN,249.38607,5.628829,NaN,NaN,3.436093,NaN,NaN,NaN,26.637485,19.733255,1554.911197,4.059474e+04
std,NaN,NaN,7.579740,NaN,109.19187,1.780559,NaN,NaN,0.881315,NaN,NaN,NaN,8.863001,8.987798,1441.855347,6.010910e+04
min,NaN,NaN,1990.000000,NaN,55.00000,0.000000,NaN,NaN,2.000000,NaN,NaN,NaN,12.000000,7.000000,2.000000,2.000000e+03
25%,NaN,NaN,2007.000000,NaN,170.00000,4.000000,NaN,NaN,2.000000,NaN,NaN,NaN,22.000000,16.000000,549.000000,2.100000e+04
50%,NaN,NaN,2015.000000,NaN,227.00000,6.000000,NaN,NaN,4.000000,NaN,NaN,NaN,26.000000,18.000000,1385.000000,2.999500e+04
75%,NaN,NaN,2016.000000,NaN,300.00000,6.000000,NaN,NaN,4.000000,NaN,NaN,NaN,30.000000,22.000000,2009.000000,4.223125e+04


In [78]:
#Removing the NAN Values
data.dropna(inplace=True)
data.reset_index(inplace=True)
data.isnull().sum()

index                0
Make                 0
Model                0
Year                 0
Engine Fuel Type     0
Engine HP            0
Engine Cylinders     0
Transmission Type    0
Driven_Wheels        0
Number of Doors      0
Market Category      0
Vehicle Size         0
Vehicle Style        0
highway MPG          0
city mpg             0
Popularity           0
MSRP                 0
dtype: int64

### Preprocessing the by data by removing stopwords and tokenizing 

In [79]:

stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    return tokens

data['Market Category'] = data['Market Category'].apply(preprocess)

data['Market Category']


0             [factory, tuner, luxury]
1                [luxury, performance]
2                             [luxury]
3                [luxury, performance]
4                             [luxury]
                     ...              
8079    [crossover, hatchback, luxury]
8080    [crossover, hatchback, luxury]
8081    [crossover, hatchback, luxury]
8082    [crossover, hatchback, luxury]
8083                          [luxury]
Name: Market Category, Length: 8084, dtype: object

###  Perform the bag-of-words approach using CountVectorizer

In [80]:
count_vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
bow = count_vectorizer.fit_transform(data['Market Category'])
print(bow.toarray()[:5, :10])

[[0 0 0 1 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 1 0]]


* We can also calculate the normalized count occurrence by dividing the count by the length of each document

In [81]:
normalized_count = bow.copy()
for i, j in zip(*normalized_count.nonzero()): 
    normalized_count[i, j] = normalized_count[i, j] / len(data['Market Category'][i])
print(normalized_count.toarray()[:5, :10]) # Print the first 5 rows and 10 columns of the matrix


[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]]


### TF-IDF to calculate the importance of each word in the document

In [82]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)
tfidf = tfidf_vectorizer.fit_transform(data['Market Category'])
print(tfidf.toarray()[:5, :10]) # Print the first 5 rows and 10 columns of the matrix


[[0.         0.         0.         0.6616931  0.         0.
  0.         0.         0.35259679 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.62784336 0.77833972]
 [0.         0.         0.         0.         0.         0.
  0.         0.         1.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.62784336 0.77833972]
 [0.         0.         0.         0.         0.         0.
  0.         0.         1.         0.        ]]


### Creating word embeddings using Word2Vec

In [83]:
model = Word2Vec(data['Market Category'], min_count=1)
embeddings = np.zeros((len(data), 100))
for i, tokens in enumerate(data['Market Category']):
    for token in tokens:
        embeddings[i] += model.wv[token]
    embeddings[i] /= len(tokens)


C:\Users\gv7lo\AppData\Local\Temp\ipykernel_6352\3049100346.py:6: RuntimeWarning: invalid value encountered in true_divide
  embeddings[i] /= len(tokens)


In [84]:
embeddings

array([[-0.01106373,  0.00585934,  0.01540476, ..., -0.03831561,
         0.01001132,  0.00379507],
       [-0.01955247,  0.00990319,  0.01476101, ..., -0.03403803,
         0.00382115,  0.00643379],
       [-0.0184865 ,  0.00986475,  0.01654671, ..., -0.04207258,
         0.0108053 ,  0.00743209],
       ...,
       [-0.01277368,  0.00704898,  0.00513679, ..., -0.02371885,
         0.00932181, -0.00200806],
       [-0.01277368,  0.00704898,  0.00513679, ..., -0.02371885,
         0.00932181, -0.00200806],
       [-0.0184865 ,  0.00986475,  0.01654671, ..., -0.04207258,
         0.0108053 ,  0.00743209]])

* We have successfully created bag-of-words vectors, normalized count occurrence vectors, TF-IDF vectors, and word embeddings for the dataset.